In [1]:
import multicomponent_evolution as mce
from scipy import spatial, cluster
import numpy as np

In [22]:
def find_chi(n_comps = 3):
    n_phases = 0
    while n_phases < n_comps:
        chi = mce.random_interaction_matrix(num_comp = n_comps)
        phis_initial = mce.get_uniform_random_composition(num_phases = 3*n_comps, num_comps = n_comps)
        phis_final = mce.evolve_dynamics(chis = chi, phis_init = phis_initial)
        n_phases = mce.count_phases(phis_final)
    return chi
np.savetxt("chi.csv",find_chi(n_comps = 2),delimiter=",")
c = np.loadtxt("chi.csv",delimiter=",")
print(c)

[[0.        5.0661078]
 [5.0661078 0.       ]]


In [20]:
n_components = 2
n_phases_guess = 6
chi = np.loadtxt('chi.csv',delimiter=",")
phis_initial = mce.get_uniform_random_composition(num_phases = n_phases_guess, num_comps = n_components)

phis_final = mce.evolve_dynamics(chis = chi, phis_init = phis_initial)
print("Number of Phases: ", mce.count_phases(phis_final))
print(phis_final)

Number of Phases:  2
[[0.51047359 0.06375321]
 [0.06376771 0.51044631]
 [0.06376771 0.51044631]
 [0.06376771 0.51044631]
 [0.06376771 0.51044631]
 [0.06376771 0.51044631]]


In [15]:
n_components = 2
n_phases_guess = 6
chi = np.loadtxt('chi.csv',delimiter=",")
phis_initial = mce.get_uniform_random_composition(num_phases = n_phases_guess, num_comps = n_components)

phis_final = mce.evolve_dynamics(chis = chi, phis_init = phis_initial)
print("Number of Phases: ", mce.count_phases(phis_final))
print(phis_final)


Number of Phases:  2
[[0.39441307 0.10025438]
 [0.10010673 0.39464814]
 [0.39441307 0.10025438]
 [0.10010673 0.39464814]
 [0.39441306 0.10025438]
 [0.39441307 0.10025438]]


In [6]:

n_components = 3
n_phases_guess = 4
chi = np.loadtxt('data/chi.csv',delimiter=",")
phis_initial = mce.get_uniform_random_composition(num_phases = n_phases_guess, num_comps = n_components)

phis_final = mce.evolve_dynamics(chis = chi, phis_init = phis_initial)
print(mce.count_phases(phis_final))
print(phis_final)
for phi in phis_final:

    print(mce.calc_diffs(phis = phi, chis = chi))
from scipy import spatial, cluster

dists = spatial.distance.pdist(phis_final)
links = cluster.hierarchy.linkage(dists, method="centroid")
clusters = cluster.hierarchy.fcluster(links,1e-2, criterion="distance")
n_clusters = len(set(clusters))

phis_out = [[] for _ in range(n_clusters)]
for i, c in enumerate(clusters):
    phis_out[c-1].append(phis_final[i])
for i, phi in enumerate(phis_out):
    phis_out[i] = np.mean(phi, axis=0)
print(phis_out)
for phi in phis_out:

    print(mce.calc_diffs(phis = phi, chis = chi))
# phis_test = mce.evolve_dynamics(chis = chi, phis_init = phis_out, tol = 1e-8)
# print(phis_test)

FileNotFoundError: data/chi.csv not found.

In [3]:
@njit
def evolution_rate(phis: np.ndarray, chis: np.ndarray = None) -> np.ndarray:
    """calculates the evolution rate of a system with given interactions

    Args:
        phis: The composition of all phases
        chis: The interaction matrix

    Returns:
        The rate of change of the composition (Eq. 4)
    """
    num_phases, num_comps = phis.shape

    # get chemical potential and pressure for all components and phases
    mus = np.empty((num_phases, num_comps))
    ps = np.empty(num_phases)
    for n in range(num_phases):  # iterate over phases
        mu, p = calc_diffs(phis[n], chis)
        mus[n, :] = mu
        ps[n] = p

    # calculate rate of change of the composition in all phases
    dc = np.zeros((num_phases, num_comps))
    for n in range(num_phases):
        for m in range(num_phases):
            delta_p = ps[n] - ps[m]
            for i in range(num_comps):
                delta_mu = mus[m, i] - mus[n, i]
                dc[n, i] += phis[n, i] * (phis[m, i] * delta_mu - delta_p)
    return dc

phis_test = mce.evolve_dynamics(chis = chi, phis_init = phis_out)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'shape' of type reflected list(array(float64, 1d, C))<iv=None>

File "multicomponent_evolution.py", line 154:
def evolution_rate(phis: np.ndarray, chis: np.ndarray = None) -> np.ndarray:
    <source elided>
    """
    num_phases, num_comps = phis.shape
    ^

During: typing of get attribute at /home/shabeeb/Projects/zwicker/multicomponent_evolution.py (154)

File "multicomponent_evolution.py", line 154:
def evolution_rate(phis: np.ndarray, chis: np.ndarray = None) -> np.ndarray:
    <source elided>
    """
    num_phases, num_comps = phis.shape
    ^

During: resolving callee type: type(CPUDispatcher(<function evolution_rate at 0x7f920358dda0>))
During: typing of call at /home/shabeeb/Projects/zwicker/multicomponent_evolution.py (188)

During: resolving callee type: type(CPUDispatcher(<function evolution_rate at 0x7f920358dda0>))
During: 

In [13]:
print(phis_test)

[[0.74023381 0.11519325 0.0334216 ]
 [0.02286568 0.02141713 0.86559559]
 [0.74023381 0.11519325 0.0334216 ]
 [0.11404217 0.74429302 0.03129822]]


In [12]:

dists = spatial.distance.pdist(phis_final)
links = cluster.hierarchy.linkage(dists, method="centroid")
clusters = cluster.hierarchy.fcluster(links, CLUSTER_DISTANCE, criterion="distance")
print(clusters)

[1 1 1 1 1 1 2 1 1 1]
